# Data Structures on the system

### Prerequisites



Run these commands in the terminal pane of your VSCode session in CodeSpaces.
Not needed if running this notebook in Binder.

In [ ]:
{
apt-get update
apt-get install -y wamerican-insane file
} >& /dev/null
dpkg -l | awk '$2 ~ /^wamerican-insane|^file/'


In [ ]:
commands="tree find ls cat head tail wc fgrep cut seq shuf sed awk"
<<< ${commands} tr ' ' '\n' | wc -l
which ${commands} | wc -l

The system has a number of data structures that it uses.  The two that we will be exploring are these:
- the file system
- a file

And like any other data structure, we can perform the four CRUD operations on them: Create, Read, Update, Delete.

### The file system

The file system is where all data is stored and organized.  The structure is that of a hierarchy or tree with branches eminating from a "root" and from "nodes" ( aka branch points ) and terminating in "leaves".  


### A file

A file is just a series of bytes of a finite length stored on the file system.

## The file system

One command that we can use to display the file system hierarch is `tree`.

For example:

In [ ]:
tree -F /etc/apt/

In [ ]:
find /etc/apt


( We will explain the details of using `tree` and other commands later.  For now, you can just run the commands to see what ouput they generate. )

Here we see a portion of the file system hierarchy.  The tree is made up of only two elements: directories ( aka folders ) and files.  Directories can contain other directories ( aka subdirectories ) and files.  In the above `tree` example, directories are denoted by ending with a '/' symbol.

Notice that the starting directory is prefixed with a slash '/'.  This is the symbol for the root of the tree or the "root" directory. The slash is also the symbol used to delimit directories from subdirectories.  For example, we can list the files and folders in a specific directory by using the `ls` command:

In [ ]:
ls -1F /etc/apt/

A few items to note:
1. all subdirectories end with a slash '/'. 
1. the contents of those subfolders are not listed
1. files are listed in alphabetical order
1. file names can have a mix of upper and lower case characters

Chaining together directories and subdirectories is called creating a path.  In the previous example '/etc/apt/' is the path. Since trusted.gpg.d is a subdirectory, we can append it to the path.  For example:

In [ ]:
ls -1F /etc/apt/trusted.gpg.d

Note there are three files in the trusted.gpg.d subdirectory and there are no further subdirectories.  We have reached the end of the path.

## Files

### File contents

Just like `ls` displays the contents of a subdirectory, `cat` displays the contents of a file ... with some interpretation.  For example:

In [ ]:
cat /etc/debian_version

`cat` displays the characters that are in the file. ( Notice the absence of a period at the end. )  However ...

What is really on the storage device is actually just a series of 1's and 0's.  And we can display that using the `xxd` command.  For example,

In [ ]:
xxd -b -g0 /etc/debian_version | cut -d' ' -f2 | tr -d '\n' ; echo 

We can modify the previous command to dispay the interpretation:

In [ ]:
xxd -b -g0 /etc/debian_version | cut -d' ' -f2-

We can modify it further to format the 1's and 0's into groups of eight, called a byte, so that they are easier to view.

In [ ]:
xxd -b -g1 /etc/debian_version | cut -d' ' -f2-

On the far right of the output, we see that the 1's and 0's that make a byte get interpreted as the letters 'b', 'u', 'l', etc.  Also notice that the last byte '00001010' was displayed as a '.' That byte is actually a non-printable character, one of many.  This one happens to be the end-of-line ( aka newline or '\n' ) character, which we will encounter more of later on.  Other non-printable characters frequently encountered include tab ( '\t' ) , carriage return ( '\r' ), and null ( '\0' ).  Lastly,the 1's and 0's could be represented in a more compact form know as hexadecimal.


In [ ]:
hexdump -Cc /etc/debian_version | cut -c9-60 | sed -e '1s/  / /g ; 2s/^ // ; 2s/   /  /g'

That is, binary '01100010' can be represented as '62' in hexadecimal.  Both of those map to the letter 'b'.  See the [ASCII man page]( https://man7.org/linux/man-pages/man7/ascii.7.html ) for a complete map.

So, what is a file?  It is a stream of 1's and 0's ( bits ) that then get grouped into chunks of 8 bits to form bytes
that are then interpreted as characters.  It's a tad more complicated than that, but pretty close for now.

### Lines

One way to interpret a file stream is as a collection of "lines" with each line being a collection of "text" characters.  Many files use this approach, e.g. CSV, YAML, HTML.  For these "text" files, a program can read the file one character at a time until it gets to an "end-of-line" character, then it can operate on that line, then read the next line.  `cat` does this with every line in a file. For example, we can have `cat` prefix each line with the line number:

In [ ]:
cat -n /etc/debian_version

That's not very exciting with a file that has only one line.  So, here's the same command run on a file with multiple lines, first without numbering the lines.

In [ ]:
cat /etc/os-release

... and again with numbering the lines.

In [ ]:
cat -n /etc/os-release

In this case, `cat` reads each character until it gets to the end of the line, prints the line number followed by the line, then repeats the process until it gets to the end of the file.

Many other commands use this pattern of reading a line, operate on it, then repeat.  Let's look at a few:
- head
- tail
- cut
- wc
- file

BTW, for all these commands, much more details on options and how they work can be found using a Google search for "unix man " followed by the command.  For example, "[unix man head](https://www.google.com/search?q=unix+man+head)"

In [ ]:
# head displays the first 10 lines of a file if not given any options
## you can specify more or fewer lines by giving it the option -n X, where X is a whole number
## here we get the first 4 words from a dictionary file
head -n 4 /usr/share/dict/words

In [ ]:
# tail displays the last 10 lines of a file if not given any options
## you can specify more or fewer lines by giving it the option -n X, where X is a whole number
## here we get the last 4 words from a dictionary file
tail -n 4 /usr/share/dict/words

In [ ]:
# cut displays the character range specified by the -c option or a field range specified by the -f option
## range is specified using 1-based counting
## here we get the first 6 characters from the /etc/debian_version file.
cat /etc/debian_version
cut -c 1-6 /etc/debian_version

In [ ]:
## here we get characters 7-10 from the /etc/debian_version file.
cut -c 7-10 /etc/debian_version

In [ ]:
# wc gives a summary of how many lines, words, and characters there are in a file
wc /usr/share/dict/words

In [ ]:
# file gives you a reasonable guess as to what type of file it is.
file /usr/share/dict/words
file /etc/dictionaries-common/words
file /usr/share/dict/american-english-insane
file /etc/debian_version
file /bin/grep

In [ ]:
cp /bin/grep /tmp/really.cool.pdf


In [ ]:
ls -la /tmp/really.cool.pdf

The caveat is that these programs only work if the file is organized as a "text" file.  That is, the bytes are interpreted as alpha-numeric characters with line endings.  When some other convention is used, then the file is termed a "binary" file. This can lead to some confusion as all files are "binary".  The difference is in how the bytes are organized in the file and interpreted by some program.

## Creating a file

The previous examples used pre-existing files.  Now we will use some commands that will create data and then put them into a file. We will explore the following commands:
- date
- echo
- seq
- curl

In [ ]:
# date prints the date
date

We can tell a command to put the data into a file by redirecting its output.  That is done using the '>' symbol.  For example, to save the output from the `date` to a file called `date.txt`:

In [ ]:
# show that date.txt does not exist
ls -F

In [ ]:
# generate a date and redirect the output to the date.txt file
date > date.txt


In [ ]:
# show that date.txt now does exist
ls -F

In [ ]:
# display the contents of the date.txt file with a line number
cat -n date.txt

In [ ]:
# echo displays the provided text
echo 'Hello, world!'

In [ ]:
# to save output to a file
echo 'Hello, world!' > hw.txt

In [ ]:
# display the contents
cat -n hw.txt

In [ ]:
ls -F

In [ ]:
# seq generates a range of numbers
seq 10 10 100 > seq.txt
cat -n seq.txt

In [ ]:
seq --help

In [ ]:
# curl GETs a webpage
## here it downloads a file containing air quality data from the city of Albuquerque
curl -s https://data.cabq.gov/airquality/aqindex/history/042222.0017 > abq.air-quality.dat
head abq.air-quality.dat


## Command pipeline

Much like one can do method chaining in Python, Ruby, JavaScript, and other languages, commands can be piped together using a vertical bar '|'.  In this way, the output of one command can be piped as input into the next command.  For example:

In [ ]:
# here the first ten lines of a file are numbered
head abq.air-quality.dat | cat -n

In [ ]:
# here only the first field is displayed from the first ten lines and then numbered
head abq.air-quality.dat | cut -d, -f 1 | cat -n

In [ ]:
# shuffle the numbered lines and show only 10, i.e. randomly pick 10 lines
cat -n abq.air-quality.dat | shuf -n 10

In [ ]:
# randomly pick 100 words, cut the first 10 characters, show the first 10, and number them
shuf -n 100 /usr/share/dict/words | cut -c1-10 | head | cat -n


### Example using transaction data

In [ ]:
url="https://ddc-datascience.s3.amazonaws.com/Projects/Project.1-Transactions/Data/Transaction.train.big.csv"
echo "${url}"


In [ ]:
curl -s "${url}" |
head -1 |
tr , '\n' |
cat -n


In [ ]:
curl -s "${url}" |
head -c-100|
file -


In [ ]:
curl -s -I "${url}" 
